In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ccxt
import yfinance as yf
from typing import Dict, Optional

class CryptoDataFetcher:
    def __init__(self):
        self.binance = ccxt.binance()
        
    def fetch_binance_data(self, symbol: str, timeframe: str = '1d', 
                          since: Optional[str] = None, limit: int = 1000) -> pd.DataFrame:
        """
        Fetch data from Binance
        symbol: e.g., 'BTC/USDT'
        timeframe: '1m', '5m', '1h', '1d', etc.
        since: 'YYYY-MM-DD'
        """
        try:
            if since:
                since = int(datetime.strptime(since, '%Y-%m-%d').timestamp() * 1000)
            
            ohlcv = self.binance.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            return df
            
        except Exception as e:
            print(f"Error fetching from Binance: {e}")
            return pd.DataFrame()

    def fetch_coinlore_data(self, coin_id: int) -> pd.DataFrame:
        """
        Fetch data from CoinLore API
        coin_id: e.g., 90 for Bitcoin
        """
        try:
            url = f"https://api.coinlore.net/api/ticker/?id={coin_id}"
            response = requests.get(url)
            data = response.json()
            return pd.DataFrame(data)
            
        except Exception as e:
            print(f"Error fetching from CoinLore: {e}")
            return pd.DataFrame()

    def fetch_yahoo_finance_data(self, symbol: str, start: str, end: str) -> pd.DataFrame:
        """
        Fetch data from Yahoo Finance
        symbol: e.g., 'BTC-USD'
        start/end: 'YYYY-MM-DD'
        """
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start, end=end)
            return df
            
        except Exception as e:
            print(f"Error fetching from Yahoo Finance: {e}")
            return pd.DataFrame()

    def fetch_coingecko_data(self, coin_id: str, vs_currency: str = 'usd', 
                            days: str = 'max') -> pd.DataFrame:
        """
        Fetch data from CoinGecko API
        coin_id: e.g., 'bitcoin'
        vs_currency: e.g., 'usd', 'eur'
        days: '1', '14', '30', '90', '180', '365', 'max'
        """
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
            params = {
                'vs_currency': vs_currency,
                'days': days,
                'interval': 'daily'
            }
            response = requests.get(url, params=params)
            data = response.json()
            
            # Create DataFrame from price data
            df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Add volume data
            df['volume'] = [x[1] for x in data['total_volumes']]
            
            return df
            
        except Exception as e:
            print(f"Error fetching from CoinGecko: {e}")
            return pd.DataFrame()

    def save_to_csv(self, df: pd.DataFrame, filename: str) -> None:
        """Save the data to a CSV file"""
        df.to_csv(filename, index=True)
        print(f"Data saved to {filename}")

# Example usage
def main():
    fetcher = CryptoDataFetcher()
    
    # Example 1: Fetch Bitcoin data from Binance
    btc_binance = fetcher.fetch_binance_data(
        symbol='BTC/USDT',
        timeframe='1d',
        since='2023-01-01'
    )
    fetcher.save_to_csv(btc_binance, 'btc_binance_data.csv')
    
    # Example 2: Fetch Bitcoin data from Yahoo Finance
    btc_yahoo = fetcher.fetch_yahoo_finance_data(
        symbol='BTC-USD',
        start='2023-01-01',
        end='2024-12-28'
    )
    fetcher.save_to_csv(btc_yahoo, 'btc_yahoo_data.csv')
    
    # Example 3: Fetch Bitcoin data from CoinGecko
    btc_coingecko = fetcher.fetch_coingecko_data(
        coin_id='bitcoin',
        days='365'
    )
    fetcher.save_to_csv(btc_coingecko, 'btc_coingecko_data.csv')

if __name__ == "__main__":
    main()